In [75]:
import sys
sys.path.append('../')

In [76]:
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine, MetaData, Table, text

from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import POSTGRES_UTEA

In [77]:
USER_DB = POSTGRES_UTEA['USER']
PASS_DB = POSTGRES_UTEA['PASSWORD']
HOST_DB = POSTGRES_UTEA['HOST']
PORT_DB = POSTGRES_UTEA['PORT']
NAME_DB = POSTGRES_UTEA['DATABASE']

ENGINE = create_engine(f'postgresql+psycopg://{USER_DB}:{PASS_DB}@{HOST_DB}:{PORT_DB}/{NAME_DB}')

PATH_OUTPUT = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\ESTIMATIVAS\V06'

metadata = MetaData()

In [78]:
# obtiene un resumen de la capa de estimativa (idd - area)
def get_resumen_estimativa():
    try:
        query = "select cast(unidad_01 as text) || '|' || cast(unidad_03 as text) || '|' || unidad_05 as idd, sum(area) as area from catastro_iag.estimativa group by idd"
        df = pd.read_sql(query, ENGINE)
        return df
    except Exception as e:
        print("❌ Error al leer datos de COSECHA 2025:", e)
        return pd.DataFrame()
    return None

# obtiene un resumen de la capa de estimativa (idd - area)
def get_resumen_cosecha():
    try:
        query = "select cast(unidad_01 as text) || '|' || cast(unidad_03 as text) || '|' || unidad_05 as idd, sum(area) as area from catastro_iag.cosecha_2025 group by idd"
        df = pd.read_sql(query, ENGINE)
        return df
    except Exception as e:
        print("❌ Error al leer datos de COSECHA 2025:", e)
        return pd.DataFrame()
    return None

# obtiene un resumen de la capa de estimativa (idd - area)
def get_resumen_estimativa():
    try:
        query = "select unidad_01, unidad_03, unidad_05, sum(area) as area from catastro_iag.estimativa group by unidad_01, unidad_03, unidad_05"
        df = pd.read_sql(query, ENGINE)
        return df
    except Exception as e:
        print("❌ Error al leer datos de COSECHA 2025:", e)
        return pd.DataFrame()
    return None

# obtiene un resumen de la capa de estimativa (idd - area)
def get_resumen_cosecha():
    try:
        query = "select unidad_01, unidad_03, unidad_05, sum(area) as area from catastro_iag.cosecha_2025 group by unidad_01, unidad_03, unidad_05"
        df = pd.read_sql(query, ENGINE)
        return df
    except Exception as e:
        print("❌ Error al leer datos de COSECHA 2025:", e)
        return pd.DataFrame()
    return None

def eliminar_propiedad_estimativa_db(cod_prop):
    """
    Elimina registros con el cod_prop enviado de la tabla estimativa
    """

    sql_query = text(f"""
        DELETE FROM catastro_iag.estimativa
        WHERE unidad_01 = :cod_prop
    """)

    try:
        with ENGINE.connect() as conn:
            result = conn.execute(sql_query, {"cod_prop": cod_prop})
            conn.commit()  # Asegura que se guarden los cambios
        print(f"Se eliminaron {result.rowcount} registros ESTIMATIVAS de la propiedad: {cod_prop}")
        return result.rowcount
    except Exception as e:
        print(f"Error al ejecutar la eliminación propiedad de ESTIMATIVAS: {e}")
        return None
    
def eliminar_propiedad_SHP_COSECHA_db(cod_prop):
    """
    Elimina registros con el cod_prop enviado de la tabla shp_cosecha
    """

    sql_query = text(f"""
        DELETE FROM catastro_iag.cosecha_2025 
        WHERE unidad_01 = :cod_prop
    """)

    try:
        with ENGINE.connect() as conn:
            result = conn.execute(sql_query, {"cod_prop": cod_prop})
            conn.commit()  # Asegura que se guarden los cambios
        print(f"Se eliminaron {result.rowcount} registros de SHP_COSECHA de la propiedad: {cod_prop}")
        return result.rowcount
    except Exception as e:
        print(f"Error al ejecutar la eliminación de SHP COSECHA: {e}")
        return None

In [79]:
df_estimativa = get_resumen_estimativa()
df_estimativa['idd'] = df_estimativa['unidad_01'].astype(str) + '|' + df_estimativa['unidad_03'].astype(str) + '|' + df_estimativa['unidad_05']
df_estimativa = df_estimativa[['idd', 'area']]
output_path = "filas_estimativas.xlsx"
df_estimativa.to_excel(output_path, index=False)
df_estimativa

,idd,area
0,1|388|L1,3.760333
1,1|388|L10.2,7.831933
2,1|388|L11,8.721908
3,1|388|L13.2,7.381695
4,1|388|L13.3,1.618728
...,...,...
10546,2302|1447|L5,16.220018
10547,2302|1447|L6,11.651393
10548,2302|1447|L7,16.645492
10549,2304|4915|L1,6.824405


In [80]:
df_cosecha = get_resumen_cosecha()
df_cosecha['idd'] = df_cosecha['unidad_01'].astype(str) + '|' + df_cosecha['unidad_03'].astype(str) + '|' + df_cosecha['unidad_05']
df_cosecha = df_cosecha[['idd', 'area']]
output_path = "filas_cosecha.xlsx"
df_cosecha.to_excel(output_path, index=False)
df_cosecha

,idd,area
0,830|1686|L1,2.356509
1,1695|40251|L1,19.618234
2,632|6578|L1,6.776382
3,2006|12314|L1,3.023093
4,2166|40185|L2,2.358242
...,...,...
10605,320|13123|L12.2,0.571402
10606,326|40141|L7.1.2,1.579672
10607,1854|13658|A3,2.531835
10608,855|12118|L9,0.865902


In [81]:
df_merged = pd.merge(
    df_estimativa,
    df_cosecha,
    on="idd",
    how="outer",
    suffixes=("_estimativa", "_cosecha")
)

In [82]:
df_merged

,idd,area_estimativa,area_cosecha
0,1|388|L1,3.760333,3.688450
1,1|388|L10.2,7.831933,7.958665
2,1|388|L11,8.721908,8.628753
3,1|388|L13.2,7.381695,7.261848
4,1|388|L13.3,1.618728,1.775559
...,...,...,...
10605,1063|16111|L12,NaN,1.084483
10606,1063|16111|L5,NaN,1.297229
10607,1507|2724|L2.1,NaN,5.037963
10608,1769|42066|L5,NaN,0.597771


In [83]:
df_merged['dif'] = abs(df_merged['area_estimativa'] - df_merged['area_cosecha'])

In [84]:
df_merged

,idd,area_estimativa,area_cosecha,dif
0,1|388|L1,3.760333,3.688450,0.071883
1,1|388|L10.2,7.831933,7.958665,0.126732
2,1|388|L11,8.721908,8.628753,0.093154
3,1|388|L13.2,7.381695,7.261848,0.119846
4,1|388|L13.3,1.618728,1.775559,0.156831
...,...,...,...,...
10605,1063|16111|L12,NaN,1.084483,NaN
10606,1063|16111|L5,NaN,1.297229,NaN
10607,1507|2724|L2.1,NaN,5.037963,NaN
10608,1769|42066|L5,NaN,0.597771,NaN


In [85]:
output_path = "filas_all.xlsx"
df_merged.to_excel(output_path, index=False)

In [86]:
df_nan_mas = df_merged[
    (df_merged["area_estimativa"].isna()) |
    (df_merged["area_cosecha"].isna()) |
    (df_merged["dif"] > 2)
]
df_nan_mas

,idd,area_estimativa,area_cosecha,dif
76,10|40256|L23.3,53.015003,50.715613,2.299390
378,30|41594|EP-L19,35.935877,38.414595,2.478718
399,30|41594|ER-L23,66.239750,59.020237,7.219513
633,55|14558|L13.2,25.446208,23.274486,2.171722
1037,114|41970|L2.2,44.158174,40.951706,3.206468
...,...,...,...,...
10605,1063|16111|L12,NaN,1.084483,NaN
10606,1063|16111|L5,NaN,1.297229,NaN
10607,1507|2724|L2.1,NaN,5.037963,NaN
10608,1769|42066|L5,NaN,0.597771,NaN


In [87]:
output_path = "filas_nan.xlsx"
df_nan_mas.to_excel(output_path, index=False)

In [88]:
df_nan_mas

,idd,area_estimativa,area_cosecha,dif
76,10|40256|L23.3,53.015003,50.715613,2.299390
378,30|41594|EP-L19,35.935877,38.414595,2.478718
399,30|41594|ER-L23,66.239750,59.020237,7.219513
633,55|14558|L13.2,25.446208,23.274486,2.171722
1037,114|41970|L2.2,44.158174,40.951706,3.206468
...,...,...,...,...
10605,1063|16111|L12,NaN,1.084483,NaN
10606,1063|16111|L5,NaN,1.297229,NaN
10607,1507|2724|L2.1,NaN,5.037963,NaN
10608,1769|42066|L5,NaN,0.597771,NaN


In [89]:
df_split = df_nan_mas.copy()
df_split[['unidad_01', 'unidad_03', 'unidad_05']] = df_split['idd'].str.split('|', expand=True)

In [90]:
df_split

,idd,area_estimativa,area_cosecha,dif,unidad_01,unidad_03,unidad_05
76,10|40256|L23.3,53.015003,50.715613,2.299390,10,40256,L23.3
378,30|41594|EP-L19,35.935877,38.414595,2.478718,30,41594,EP-L19
399,30|41594|ER-L23,66.239750,59.020237,7.219513,30,41594,ER-L23
633,55|14558|L13.2,25.446208,23.274486,2.171722,55,14558,L13.2
1037,114|41970|L2.2,44.158174,40.951706,3.206468,114,41970,L2.2
...,...,...,...,...,...,...,...
10605,1063|16111|L12,NaN,1.084483,NaN,1063,16111,L12
10606,1063|16111|L5,NaN,1.297229,NaN,1063,16111,L5
10607,1507|2724|L2.1,NaN,5.037963,NaN,1507,2724,L2.1
10608,1769|42066|L5,NaN,0.597771,NaN,1769,42066,L5


In [91]:
lista_cods = list(set(df_split['unidad_01']))
lista_cods = [int(i) for i in lista_cods]
lista_cods.sort()
print('Cantidad:', len(lista_cods))
print(lista_cods)

Cantidad: 25
[10, 30, 55, 110, 114, 122, 217, 239, 275, 300, 303, 305, 326, 480, 482, 592, 629, 949, 1052, 1063, 1314, 1454, 1507, 1769, 2189]


In [92]:
len(list(set(df_split['unidad_01'])))

25

In [55]:
for i in lista_cods:
    eliminar_propiedad_estimativa_db(i)

Se eliminaron 8090 registros ESTIMATIVAS de la propiedad: 10
Se eliminaron 2920 registros ESTIMATIVAS de la propiedad: 13
Se eliminaron 18158 registros ESTIMATIVAS de la propiedad: 17
Se eliminaron 11592 registros ESTIMATIVAS de la propiedad: 19
Se eliminaron 6220 registros ESTIMATIVAS de la propiedad: 23
Se eliminaron 11136 registros ESTIMATIVAS de la propiedad: 29
Se eliminaron 17428 registros ESTIMATIVAS de la propiedad: 30
Se eliminaron 12958 registros ESTIMATIVAS de la propiedad: 31
Se eliminaron 6293 registros ESTIMATIVAS de la propiedad: 33
Se eliminaron 8654 registros ESTIMATIVAS de la propiedad: 47
Se eliminaron 6082 registros ESTIMATIVAS de la propiedad: 55
Se eliminaron 8374 registros ESTIMATIVAS de la propiedad: 92
Se eliminaron 1503 registros ESTIMATIVAS de la propiedad: 110
Se eliminaron 3962 registros ESTIMATIVAS de la propiedad: 113
Se eliminaron 14492 registros ESTIMATIVAS de la propiedad: 114
Se eliminaron 12058 registros ESTIMATIVAS de la propiedad: 122
Se eliminaron

In [56]:
resumen_estimativa = get_resumen_estimativa()
resumen_cosecha = get_resumen_cosecha()

print('Total Resumen Estimativa:', resumen_estimativa['area'].sum())
print('Total Resumen SHPCosecha:', resumen_cosecha['area'].sum())

Total Resumen Estimativa: 37476.39200378607
Total Resumen SHPCosecha: 55483.70074953324


In [15]:
# identifiar lotes que estan en ESTIMATIVAS y no en SHP_Cosecha
faltantes = resumen_estimativa[~resumen_estimativa['idd'].isin(resumen_cosecha['idd'])].copy()
if len(faltantes) > 0:
    # Dividir la columna idd en 3 columnas nuevas ('unidad_01', 'unidad_03', 'unidad_05')
    faltantes[['unidad_01', 'unidad_03', 'unidad_05']] = faltantes['idd'].str.split('|', expand=True)
    # extraer lista de unicos de codigos de propiedad
    codigos_prop = list(set(faltantes['unidad_01']))
    # convertir lista de str a int
    codigos_prop = [int(i) for i in codigos_prop]
    codigos_prop.sort()
    codigos_prop
else:
    codigos_prop = []
    print('No existen lotes que esten en ESTIMATIVAS y no en SHP COSECHA')

No existen lotes que esten en ESTIMATIVAS y no en SHP COSECHA


In [16]:
codigos_prop

[]

In [17]:
for i in codigos_prop:
    eliminar_propiedad_db(i)

In [11]:
resumen_estimativa = get_resumen_estimativa()
resumen_cosecha = get_resumen_cosecha()

print('Total Resumen Estimativa:', resumen_estimativa['area'].sum())
print('Total Resumen SHPCosecha:', resumen_cosecha['area'].sum())

Total Resumen Estimativa: 54030.595134443785
Total Resumen SHPCosecha: 55483.70074953324


In [12]:
# identifiar lotes que estan en ESTIMATIVAS y no en SHP_Cosecha
faltantes = resumen_cosecha[~resumen_cosecha['idd'].isin(resumen_estimativa['idd'])].copy()
if len(faltantes) > 0:
    # Dividir la columna idd en 3 columnas nuevas ('unidad_01', 'unidad_03', 'unidad_05')
    faltantes[['unidad_01', 'unidad_03', 'unidad_05']] = faltantes['idd'].str.split('|', expand=True)
    # extraer lista de unicos de codigos de propiedad
    codigos_prop = list(set(faltantes['unidad_01']))
    # convertir lista de str a int
    codigos_prop = [int(i) for i in codigos_prop]
    codigos_prop.sort()
    codigos_prop
else:
    codigos_prop = []
    print('No existen lotes que esten en SHP COSEHCA y no en ESTIMATIVA')

In [13]:
codigos_prop

[62,
 123,
 141,
 205,
 408,
 504,
 560,
 592,
 630,
 673,
 773,
 782,
 877,
 1222,
 1507,
 1567,
 1656,
 1801,
 2189]

In [129]:
for i in codigos_prop:
    eliminar_propiedad_db(i)